In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [ ]:
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.1 MB 4.9 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 895 kB 39.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [ ]:
# model.generate(pieces)
from transformers import MT5Config, MT5Tokenizer, MT5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
class CustomDataset:

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:


def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

hyper-parameters

In [ ]:
config = MT5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [ ]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)


In [ ]:
!nvidia-smi


Tue Nov 23 05:27:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    35W / 250W |  16217MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/train_data.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/validation_data.csv')[['document','label']]

partial dataset


In [ ]:
import numpy as np

train_dataset = train_dataset.sample(frac=1).reset_index(drop=True).iloc[:2000]
validation_dataset = validation_dataset.sample(frac=1).reset_index(drop=True).iloc[:500]

train

In [ ]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

1


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  25.1658878326416


500it [59:27,  6.83s/it]

Epoch: 0, Loss:  3.5197627544403076


1000it [1:55:48,  6.95s/it]


1


0it [00:00, ?it/s]

Epoch: 1, Loss:  1.959478497505188


304it [34:06,  6.77s/it]

valid(test)

In [ ]:


validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200


,Generated Text,Actual Text
0,<extra_id_0> 데이터를 공유했을 때 사람들의 행동 변화를 이끌어낼 수 있다...,스마트시티 분야 권위자인 MIT 교수는 도시와 사회에 미치는 데이터의 힘을 강조하였...
1,<extra_id_0>다.,오너 일가의 상속세 납부를 위해 LG는 2017년까지 주당 배당금 1300원을 20...
2,<extra_id_0>하고 있다.,삼성전자는 교원웰스를 통해 자사의 비스포크 김치냉장고를 렌탈 판매할 예정이며 이로써...
3,"<extra_id_0>을 보고받고, A 재활용업체가 불법으로 방치한 폐기물 처리 현...",환경부 조명래 장관과 김주수 의성군수는 21일 경북 의성군 폐기물 관리법을 위반한 ...
4,<extra_id_0> 1호로 관심을 모았던 국회 수소충전소가 문을 연다.,산업통상자원부는 10일 산업융합 분야 규제 샌드박스 1호로 관심을 모았던 국회 수소...
...,...,...
495,<extra_id_0> 등 다양한 방법으로 진행된다.,홈플러스가 오는 17일까지 대규모 온라인 사은행사인 '땡Q 페스티벌'을 계획하고 할...
496,<extra_id_0> 추모 음악회'와 '김영의 선생님 사진·유물전' 개최된다.,이화여자대학교와 음악대학이 본교 창립 133주년 및 故 김영의 선생(1908~198...
497,"<extra_id_0>며 ""그런 식의 이야기를 한다""고 밝혔다.",16일 박지원 민주평화당 의원은 tbs 라디오에서 손학규 바른미래당 대표가 민평당 ...
498,<extra_id_0>하는 취지로 작년부터 매년 100개 기업을 선정했다.,LED 조명전문기업 글로우원은 8일 고용노동부가 주관하는 2019 대한민국 일자리 ...


In [34]:
for i in range(100):
  print(final_df['Generated Text'][i])

<extra_id_0> 데이터를 공유했을 때 사람들의 행동 변화를 이끌어낼 수 있다고 설명했다.
<extra_id_0>다.
<extra_id_0>하고 있다.
<extra_id_0>을 보고받고, A 재활용업체가 불법으로 방치한 폐기물 처리 현장을 살폈다.
<extra_id_0> 1호로 관심을 모았던 국회 수소충전소가 문을 연다.
<extra_id_0> 시대와 미래에 대비할 수 있는 '수원형 인구정책'을 수립하고 있다.
<extra_id_0>을 기반으로 자동 상담을 지원하는 민원서비스가 개발됐다.
<extra_id_0> 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투입하고, 중앙본부 특별점검반 1대 추가 운영 등 총 28개반을 현장에 투
<extra_id_0> 등 다양한 기후기술과 기술이 적용된 시제품, 상용제품을 선보인다.
<extra_id_0> 고장 문제로 도마에 오르고 있다.
<extra_id_0> 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위자로 약물전달시스템 분야 연구의 권위
<extra_id_0> 네트워크를 결합한 협업으로 현대글로비스가 중국을 비롯한 글로벌 사업 분야 확대를 위한 양해각서(MOU)를 체결했다고 29일 밝혔다.
<extra_id_0>'을 개최하고 우수사